In [5]:
import numpy as np
import re
import collections
vowels = list('aeioóuyąę') + list('aeioóuyąę'.upper())
compacted_vovels = ['i' + x for x in vowels if x != 'i']

In [6]:
def count_syllable(phrase, verose=False):
    res = 0
    for i, letter in enumerate(phrase):
        if letter in vowels:
            res += 1
            if verose:
                print(letter)
        if phrase[i:i+2] in compacted_vovels:
            res -= 1
            if verose:
                print(phrase[i:i+2])
    return res
def string_norm(sent, num=False):
    regex = re.compile(f'[,\.!?:;\'{"0-9" if not num else ""}\*\-“…\(\)„”—»«–––=\[\]’]')
    return regex.sub('',sent.lower())

In [7]:
%%time

bigrams =  collections.defaultdict(lambda: collections.defaultdict(int))
bigrams_rev = collections.defaultdict(lambda: collections.defaultdict(int))
unigrams = collections.defaultdict(int)
with open('../data/poleval_2grams.txt') as f:
    for x in f:
        num, w1, w2 = x.split()
        if float(num) >= 5:
            bigrams[w1][w2] = float(num)
            bigrams_rev[w2][w1] = float(num)
            unigrams[w1] += float(num) / 2
            unigrams[w2] += float(num) / 2



CPU times: user 1min 58s, sys: 2.36 s, total: 2min
Wall time: 1min 59s


In [8]:
with open('../data/supertags.txt') as f:
    supertags = [tuple(x.split()) for x in f]
    supertags = {k: v for (k,v) in supertags}
    supertags_ID = collections.defaultdict(set) 
    for k,v in supertags.items():
        supertags_ID[v].add(k)

In [9]:
with open('../data/pan-tadeusz.txt') as f:
    pt = [string_norm(x).split() for x in f if  count_syllable(string_norm(x)) == 13]
    
acents = collections.Counter([
    tuple([count_syllable(y) for y in x])
    for x in pt
])

In [10]:
def get_rym(w):
    best = None
    for i in range(len(w)):
        if count_syllable(w[i:]) == 2:
            best = w[i:]
    return best

def sample_verset():
    try:
        index = np.random.choice(np.arange(len(pt)))
        rym = get_rym(pt[index][-1])
        if get_rym(pt[index + 1][-1]) == rym:
            return pt[index], pt[index + 1]
        elif get_rym(pt[index - 1][-1]) == rym:
            return pt[index], pt[index - 1]
        return sample_verset()
    except:
        return sample_verset()
    
def get_acents(phrase):
    return [count_syllable(x) for x in phrase]

def get_ryhm_bigram(rym):
    temp = {w1: {w2: cnt for w2, cnt in v.items() if rym in w2} for w1,v in bigrams.items() if rym in w1}
    return temp



In [52]:
class EmptySample(Exception):
    pass

def sample(words_with_counters, verbose=False):
    if len(words_with_counters) == 0:
        raise EmptySample()
    if verbose:
        print('samled from:', len(words_with_counters))
    words, cnt = zip(*words_with_counters.items())
    propabilities = np.array(cnt) / np.sum(cnt)
    return np.random.choice(words, p=propabilities)
# print(sample(list({'a':1, 'b':2})))

def f():
    for i in range(10):
        p1, p2 = sample_verset()
        n1 = gen_phrase(p1[::-1])
        if n1 is None:
            continue
        n2 = gen_phrase(p2[::-1])
        if n2 is None:
            continue
        print()
        print(' '.join(p1 + p2))
        print(' '.join(n1[::-1] + n2[::-1]))
        return
def gen_phrase(phrase, rhymes=None):
    print('.', end='')
#     rhymes = 
#     np.random.seed(4)
    for ii in range(10):
        try:
            rhyme = get_rym(phrase[0])
            new_word = sample({
                k:(v) for k,v in unigrams.items() 
                if k.endswith(rhyme)
                if supertags.get(k) == supertags.get(phrase[0])
                if count_syllable(k) == count_syllable(phrase[0])
            }, False)
            aviable_bigrams = bigrams_rev[new_word]
            new_phrase = [new_word]

            for i in range(1, len(phrase)):
                word = phrase[i]
#                 print(new_phrase)
#                 print('words in bigrams_rev', len(bigrams_rev[new_word]))
#                 if not (rhymes[i] is None):
#                     print('TUTAJ' , word)
                new_word = sample({
                    k:v for k,v in bigrams_rev[new_word].items() 
                    if supertags.get(k) == supertags.get(word)
                    if count_syllable(k) == count_syllable(word)
                    
                   
                }, False)
                new_phrase.append(new_word)
                        
        except EmptySample as e:
            pass
        if len(phrase) == len(new_phrase):
                return new_phrase
#         else:
#             print(len(phrase), len(new_phrase))
f()
# mockup_phrase = sample_verset()
# # p = mockup_phrase[0] + mockup_phrase[1]
# gen_phrase(mockup_phrase[0])[::-1] + gen_phrase(mockup_phrase[1])[::-1]

...
a na głos dzwonka niby na wiatru powianie chylą się wszystkie głowy jak kłosy na łanie
się na ruch koła również do rządu pytanie sądzą że wszystkie firmy jak środki na łanie
